# Training and Testing Neural Networks using the New Optimizers

### Imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt

### Custom models and optimizers

In [ ]:
from models import SimpleCNN, SimpleMLP     # defined in models.py
from new_optimizers import MomentumSGD_Strong_Wolfe, Adam_Strong_Wolfe, TrustRegionCauchy   # defined in new_optimizers.py

## Task 1: Digit Classification on MNIST Using CNNs

### Downloading the dataset and normalizing

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)) # mean and std for MNIST
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

### Preparing PyTorch dataLoaders

In [ ]:
torch.manual_seed(42)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)

### Listing new optimizers and benchmark optimizers

In [ ]:
# Benchmarks
optimizer1 = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer2 = torch.optim.Adam(model.parameters(), lr=0.001)

# Custom optimizers
optimizer3 = MomentumSGD_Strong_Wolfe(model.parameters(), alpha1=0.001)
optimizer4 = Adam_Strong_Wolfe(model.parameters(), alpha1=0.01, betas=(0.9, 0.999))
optimizer5 = TrustRegionCauchy(model.parameters())

### Defining train and test functions

In [ ]:
criterion = nn.CrossEntropyLoss()

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    avg_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        def closure(backward=True):
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            if backward:
                loss.backward()
            return loss
        
        loss = closure()
        optimizer.step(closure)
        
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}'
                  f' ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

        avg_loss += loss.item()
    
    avg_loss /= len(train_loader)
    return avg_loss
        
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)}'
          f' ({100. * correct / len(test_loader.dataset):.0f}%)\n')
    return test_loss, correct / len(test_loader.dataset)

### Model training with the optimizers

In [ ]:
epochs = 10
all_train_loss = []
all_test_loss = []
all_test_acc = []

In [ ]:
# Benchmark 1: SGD with Momentum
print("Benchmark 1: SGD with Momentum")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleConvNN().to(device)

train_loss_over_epochs = []
test_loss_over_epochs = []
test_acc_over_epochs = []
for epoch in range(1, epochs + 1):
    avg_loss = train(model, device, train_loader, optimizer1, epoch)
    train_loss_over_epochs.append(avg_loss)
    test(model, device, test_loader)
    test_loss, test_acc = test(model, device, test_loader)
    test_loss_over_epochs.append(test_loss)
    test_acc_over_epochs.append(test_acc)

all_train_loss.append(train_loss_over_epochs)
all_test_loss.append(test_loss_over_epochs)
all_test_acc.append(test_acc_over_epochs)

In [ ]:
# Benchmark 2: Adam
print("Benchmark 2: Adam")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleConvNN().to(device)

train_loss_over_epochs = []
test_loss_over_epochs = []
test_acc_over_epochs = []
for epoch in range(1, epochs + 1):
    avg_loss = train(model, device, train_loader, optimizer2, epoch)
    train_loss_over_epochs.append(avg_loss)
    test(model, device, test_loader)
    test_loss, test_acc = test(model, device, test_loader)
    test_loss_over_epochs.append(test_loss)
    test_acc_over_epochs.append(test_acc)

all_train_loss.append(train_loss_over_epochs)
all_test_loss.append(test_loss_over_epochs)
all_test_acc.append(test_acc_over_epochs)

In [ ]:
# New Optimizer 1: MomentumSGD_Strong_Wolfe
print("New Optimizer 1: MomentumSGD_Strong_Wolfe")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleConvNN().to(device)

train_loss_over_epochs = []
test_loss_over_epochs = []
test_acc_over_epochs = []
for epoch in range(1, epochs + 1):
    avg_loss = train(model, device, train_loader, optimizer3, epoch)
    train_loss_over_epochs.append(avg_loss)
    test(model, device, test_loader)
    test_loss, test_acc = test(model, device, test_loader)
    test_loss_over_epochs.append(test_loss)
    test_acc_over_epochs.append(test_acc)

all_train_loss.append(train_loss_over_epochs)
all_test_loss.append(test_loss_over_epochs)
all_test_acc.append(test_acc_over_epochs)

In [ ]:
# New Optimizer 2: Adam_Strong_Wolfe
print("New Optimizer 2: Adam_Strong_Wolfe")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleConvNN().to(device)

train_loss_over_epochs = []
test_loss_over_epochs = []
test_acc_over_epochs = []
for epoch in range(1, epochs + 1):
    avg_loss = train(model, device, train_loader, optimizer4, epoch)
    train_loss_over_epochs.append(avg_loss)
    test(model, device, test_loader)
    test_loss, test_acc = test(model, device, test_loader)
    test_loss_over_epochs.append(test_loss)
    test_acc_over_epochs.append(test_acc)

all_train_loss.append(train_loss_over_epochs)
all_test_loss.append(test_loss_over_epochs)
all_test_acc.append(test_acc_over_epochs)

In [ ]:
# New Optimizer 3: TrustRegionCauchy
print("New Optimizer 3: TrustRegionCauchy")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleConvNN().to(device)

train_loss_over_epochs = []
test_loss_over_epochs = []
test_acc_over_epochs = []
for epoch in range(1, epochs + 1):
    avg_loss = train(model, device, train_loader, optimizer5, epoch)
    train_loss_over_epochs.append(avg_loss)
    test(model, device, test_loader)
    test_loss, test_acc = test(model, device, test_loader)
    test_loss_over_epochs.append(test_loss)
    test_acc_over_epochs.append(test_acc)

all_train_loss.append(train_loss_over_epochs)
all_test_loss.append(test_loss_over_epochs)
all_test_acc.append(test_acc_over_epochs)

In [ ]:
# Plotting MomentumSGD vs MomentumSGD_Strong_Wolfe
plt.figure(figsize=(12, 5))
plt.plot(all_train_loss[0], label='SGD with Momentum - Train Loss')
plt.plot(all_train_loss[2], label='MomentumSGD_Strong_Wolfe - Train Loss')
plt.title('Training Loss Comparison')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()